<h2> Applied Data Science Capstone </h2>
<h2> Week 3 Assignment - Segmenting and Clustering Neighborhoods in Toronto </h2>
<br>


<h3> Import libraries needed for all three parts</h3>

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import wget
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium


In [2]:
CLIENT_ID = 'V25OYZRTG244PPB512XXX5IA52HZPBVVREBOYUF2HCUxxxxx' # your Foursquare ID
CLIENT_SECRET = 'EAHMU2VCDRJBIDJ5A5JRNF4FJB102GNEX20CLIPH5F5xxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


<h2> Part 1 - Import Canadian postal codes, boroughs, and neighborhoods </h2>

<h3> Define URL and import content from URL into Beautiful Soup </h3>

In [3]:
#definte url to download content
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#download url content via request, then put content in beautifulsoup
page = requests.get(wikiurl)
soup = BeautifulSoup(page.content, 'lxml')


<h3>Extract table from beautiful soup</h3>

In [4]:
#extract table from beautifulsoup with id, then find the table, then find all fields with 'tr' heading
table = soup.find(id="mw-content-text").find('table').find_all('tr')


<h3>Extract postal codes, boroughs, and neighborhood into a data frame</h3>

In [5]:
#create an empty pandas data frame
toronto_pc = pd.DataFrame(columns = ('PostalCode','Borough','Neighborhood'))


In [6]:
#loop through each field in table, extract postal code, borough, and neighborhood and append to data frame

i = 1 #skip first field (i=0) as it only contains headings
while i != -1:
    try:
        data = table[i].find_all('td') #find all td fields
        pc = data[0].get_text().rstrip('\n') #extract 1st field - postal code
        br = data[1].get_text().rstrip('\n') #extract 2nd field - borough
        nb = data[2].get_text().rstrip('\n') #extract 3rd field - neighborhood
        if br != 'Not assigned':   #only append data frame if borough is assigned
            toronto_pc = toronto_pc.append({'PostalCode' : pc, 
                                          'Borough' : br, 
                                          'Neighborhood': nb}, 
                                         ignore_index = True)
        i += 1
    except IndexError:  #end of the table, exit loop
        i = -1


<h3>Print number of rows of data frame</h3>

In [7]:
print('There are {} rows in the data frame'.format(toronto_pc.shape[0]))

There are 103 rows in the data frame


<br><h2> Part 2 - Add latitudes and longitudes to data frame </h2>
<h3> <i><u> Latitudes and longitudes from assignment's csv file will be used here </u></i></h3>

<h3>Definte URL and import postal code csv into a pandas data frame </h3>

In [8]:
#definte url to download postal code csv
postalcodeurl = 'http://cocl.us/Geospatial_data'

#download data via wget and put it in a pandas data frame
wget.download(postalcodeurl,'postal_code.csv')
pc_table = pd.read_csv('postal_code.csv')

#rename column heading for postal code to be consistent with the first data frame
pc_table = pc_table.rename(columns={'Postal Code':'PostalCode'}) 


<h3>Merge the two tables </h3>

In [9]:
#merg toronto_pc with pc_table by matching postal code
toronto_pc = toronto_pc.merge(pc_table, on='PostalCode', how='left')

#print out first 10 records of newly merged data frame
toronto_pc.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<br><h2> Part 3 - Neighbood Analysis </h2>
<h3> <i><u> Central Toronto and Downtown Toronto neighboorhoods will be analyzed here </u></i></h3>

<h3>Create a new data frame containg data for Central Toronto and Downtown Toronto</h3>

In [10]:
#filter out Central Toronto and Downtown Toronto rows and put them in a new data frame
toronto_cd = toronto_pc[toronto_pc['Borough'].isin(['Central Toronto','Downtown Toronto'])].reset_index(drop=True)
toronto_cd.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


<h3>Find coordinates (latitude & longitude) for Toronto</h3>

In [11]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h3> Visualize the selected postal codes on a map </h3>

In [12]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map for each neighborhood
for lat, lng, label in zip(toronto_cd['Latitude'], toronto_cd['Longitude'], toronto_cd['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

#display map
map_toronto

<h3>Define a function to find nearby venues by providing neighborhood name, latitude/longitude, radius, and limit,
    and call on FourSquare API to extract venue data</3>


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius, limit):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3>Call the function above to run thru each neighborhood and place nearby venues found in a new data frame </h3>

In [15]:
LIMIT = 100 #set limit of how many top venues to retrieve
RADIUS = 750 #set radius distance to search for venues

#call function to look for nearby venues for each neighborhood
toronto_venues = getNearbyVenues(names=toronto_cd['Neighborhood'],
                                 latitudes=toronto_cd['Latitude'],
                                 longitudes=toronto_cd['Longitude'],
                                 radius=RADIUS, 
                                 limit=LIMIT)


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
University of Toronto, Harbord
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


<h3> Print the shape of the data frame and display first few rows</h3>

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(1937, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


<h3>Group the data by Neighborhood and display count of each column</h3>

In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",26,26,26,26,26,26
Central Bay Street,100,100,100,100,100,100
Christie,34,34,34,34,34,34
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,73,73,73,73,73,73
Davisville North,35,35,35,35,35,35
"First Canadian Place, Underground city",100,100,100,100,100,100


<h3>Let's find how many unique categories there are in the data</h3>

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 246 uniques categories.


<h3>Next, a new data frame is created by categorizing every row of the venue data frame from above into a matrix of venue types </h3>

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

#rename category 'Neighborhood' to 'Community' as we already use Neighborhood as a column name in many data frames
toronto_onehot = toronto_onehot.rename(columns={'Neighborhood':'Community'})

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] #use 'Neighborhood.' for 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#display first few rows
toronto_onehot.head()


,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>Displaying the size of this new data frame</h3>

In [20]:
toronto_onehot.shape

(1937, 247)

<h3>Next, this data frame is grouped by neighborhood, and each venue type gets the mean of its frequency of occurrence in that neighborhood</h3>

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,...,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.010000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.010000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.038462,0.038462,0.038462,0.076923,0.076923,0.115385,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
2,Central Bay Street,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.01,0.010000,0.01,0.000000,0.010000,0.00,0.010000,0.00,0.010000
3,Christie,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
4,Church and Wellesley,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,...,0.00,0.00,0.010000,0.00,0.010000,0.000000,0.00,0.010000,0.00,0.020000
5,"Commerce Court, Victoria Hotel",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.00,0.00,0.020000,0.00,0.000000,0.000000,0.01,0.000000,0.00,0.000000
6,Davisville,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,...,0.00,0.00,0.013699,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
7,Davisville North,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.028571,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000
8,"First Canadian Place, Underground city",0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00,0.00,0.020000,0.00,0.000000,0.000000,0.00,0.000000,0.01,0.000000
9,Forest Hill North & West,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000


<h3>Displaying the size of this grouped data frame</h3>

In [22]:
toronto_grouped.shape

(28, 247)

<h3>To review the data frame further, top 5 venue types for each neighborhood are displayed</h3>

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.05
2                 Café  0.05
3  Japanese Restaurant  0.04
4           Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0  Airport Terminal  0.12
1  Sculpture Garden  0.08
2    Airport Lounge  0.08
3   Airport Service  0.08
4   Harbor / Marina  0.08


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2  Japanese Restaurant  0.05
3          Art Gallery  0.04
4       Clothing Store  0.04


----Christie----
           venue  freq
0           Café  0.12
1  Grocery Store  0.12
2           Park  0.09
3          Diner  0.06
4    Coffee Shop  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.07
1  Japanese Restaurant  0.05
2     Sushi Restaurant  0.04
3           Restaurant  0.

<h3>Create a function to find top venue types by providing data and number of top venues to search for</h3>

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h3>Call the function above with each row of the grouped venue data frame created above and put the data into another data frame to display top 10 venue types for each neighborhood</h3>

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Beer Bar,Bakery,Cocktail Bar,Italian Restaurant,Art Gallery
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Terminal,Coffee Shop,Airport Service,Rental Car Location,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport Lounge,Park,Boutique
2,Central Bay Street,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Clothing Store,Italian Restaurant,Ramen Restaurant,Chinese Restaurant,Ice Cream Shop,Bubble Tea Shop
3,Christie,Café,Grocery Store,Park,Coffee Shop,Diner,Indian Restaurant,Italian Restaurant,Hardware Store,Athletics & Sports,Candy Store
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Men's Store,Café,Restaurant,Sandwich Place,Park,Smoke Shop,Mediterranean Restaurant


<h3>Use <i>K-Means</i> function to cluster neighborhood into 4 clusters </h3>

In [26]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3], dtype=int32)

<h3>Create a new data frame by merging the cluster labels and the top 10 venue types for each neighborhood</h3>

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_cd.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Italian Restaurant,Theater,Restaurant,Café,Pub,Bakery,Thai Restaurant,Cosmetics Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Park,Gastropub,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Japanese Restaurant,Hotel,Gastropub,Ramen Restaurant,Fast Food Restaurant,Plaza,Bubble Tea Shop,Diner,Theater
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Gastropub,Restaurant,American Restaurant,Seafood Restaurant,Theater,Italian Restaurant,Cosmetics Shop,Bakery
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Beer Bar,Bakery,Cocktail Bar,Italian Restaurant,Art Gallery


<h3>Visualize the clusters on a map</h3>

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

#display map
map_clusters

<h3>Examine data from Cluster 1</h3>

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Italian Restaurant,Theater,Restaurant,Café,Pub,Bakery,Thai Restaurant,Cosmetics Shop
1,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Park,Gastropub,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Hotel,Gastropub,Ramen Restaurant,Fast Food Restaurant,Plaza,Bubble Tea Shop,Diner,Theater
3,Downtown Toronto,0,Café,Coffee Shop,Gastropub,Restaurant,American Restaurant,Seafood Restaurant,Theater,Italian Restaurant,Cosmetics Shop,Bakery
4,Downtown Toronto,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Beer Bar,Bakery,Cocktail Bar,Italian Restaurant,Art Gallery
5,Downtown Toronto,0,Coffee Shop,Café,Japanese Restaurant,Art Gallery,Clothing Store,Italian Restaurant,Ramen Restaurant,Chinese Restaurant,Ice Cream Shop,Bubble Tea Shop
6,Downtown Toronto,0,Café,Grocery Store,Park,Coffee Shop,Diner,Indian Restaurant,Italian Restaurant,Hardware Store,Athletics & Sports,Candy Store
7,Downtown Toronto,0,Café,Hotel,Coffee Shop,Theater,Clothing Store,Restaurant,Gastropub,Gym,Cosmetics Shop,Concert Hall
8,Downtown Toronto,0,Coffee Shop,Hotel,Boat or Ferry,Brewery,Japanese Restaurant,Park,Aquarium,Baseball Stadium,Scenic Lookout,Bar
9,Downtown Toronto,0,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Concert Hall,Seafood Restaurant,Gym,Beer Bar,Monument / Landmark


<h3>Examine data from Cluster 2</h3>

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Central Toronto,1,Playground,Home Service,Spa,Pet Store,Health & Beauty Service,Garden,Event Space,Ethiopian Restaurant,Falafel Restaurant,Discount Store


<h3>Examine data from Cluster 3</h3>

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Central Toronto,2,Home Service,Swim School,Bus Line,Park,Business Service,Coffee Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Distribution Center


<h3>Examine data from Cluster 4</h3>

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Central Toronto,3,Park,Jewelry Store,Bus Line,Sushi Restaurant,Gym / Fitness Center,Trail,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop
23,Downtown Toronto,3,Park,Trail,Playground,Candy Store,Yoga Studio,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Dumpling Restaurant


<h3>Analysis Conclusion:</h3>
<p><strong>Even though the neighborhoods were clustered into 4 groups, vast majority belonged to one cluster while the other three clusters only 
    contain one or two neighborhood. This seems to indicate that neighborhood in Central Toronto and Downtown Toronto boroughs are fairly
    homogeneous. This may be due to the fact that downtown and central areas of a metropolitan city tend to be commercial/business districts;
    therefore, venues across those areas are generally similar.</strong></p>